# Goal 3: Calibration and Rectification

### Step #01: Load images 

In [9]:
import numpy as np
import cv2
import glob
import os
import datetime

In [10]:
# ---------------------------------------------------------------------------
# Constants
# ---------------------------------------------------------------------------
RELATIVE_RAW_DATASET_PATH = "../raw"
LEFT_IMG_PATH  = f"{RELATIVE_RAW_DATASET_PATH}/calib/image_02/data"
RIGHT_IMG_PATH = f"{RELATIVE_RAW_DATASET_PATH}/calib/image_03/data"

In [11]:
# Board definitions
boards = [
    {"name": "boardBig", "pattern_size": (11,7)},
    {"name": "boardMed", "pattern_size": (5,7)},
]

# VISUALIZATION SETTINGS
VISUALIZE = True
VIS_DELAY_MS = 0  # Wait 500ms between images. Set to 0 to wait for keypress.

# !!! CRITICAL !!! 
# Measure your real square size. 
# USE METERS to match KITTI format (e.g., 30mm = 0.03)
REAL_SQUARE_SIZE_M = 0.1

# Precompute object point templates
obj_templates = {}
for b in boards:
    nx, ny = b["pattern_size"]
    # We apply the real world scale here
    objp = np.zeros((nx*ny, 3), np.float32)
    objp[:, :2] = np.mgrid[0:nx, 0:ny].T.reshape(-1, 2) * REAL_SQUARE_SIZE_M
    obj_templates[b["name"]] = objp

In [12]:
import cv2
import numpy as np
import glob
import os

# ---------------------------------------------------------------------------
# 1. CONFIGURATION
# ---------------------------------------------------------------------------

RELATIVE_RAW_DATASET_PATH = "../raw"
LEFT_IMG_PATH  = f"{RELATIVE_RAW_DATASET_PATH}/calib/image_02/data"
RIGHT_IMG_PATH = f"{RELATIVE_RAW_DATASET_PATH}/calib/image_03/data"

# Physical 12x8 squares -> (11, 7) internal corners
# Physical 6x8 squares  -> (5, 7) internal corners
boards = [
    {"name": "boardBig", "pattern_size": (11, 7)},
    {"name": "boardMed", "pattern_size": (5, 7)},
]

VISUALIZE = True

# ---------------------------------------------------------------------------
# 2. PROCESSING LOGIC
# ---------------------------------------------------------------------------

def get_image_files(directory):
    if not os.path.exists(directory):
        return []
    files = sorted(glob.glob(os.path.join(directory, "*.png")))
    if not files:
        files = sorted(glob.glob(os.path.join(directory, "*.jpg")))
    return files

def sharpen_image(gray_img):
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    return cv2.filter2D(gray_img, -1, kernel)

def find_and_draw_centroids(img, display_prefix=""):
    vis_img = img.copy()
    
    gray_base = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray_processing = sharpen_image(gray_base)

    subpix_criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 40, 0.001)
    
    for board in boards:
        name = board["name"]
        nx, ny = board["pattern_size"]
        instance_count = 0
        
        while True:
            # 1. Detect (Normal Scale)
            flags = cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_NORMALIZE_IMAGE
            ret, corners = cv2.findChessboardCorners(gray_processing, (nx, ny), flags)
            
            # 2. Detect (Upscale if needed)
            if not ret:
                gray_upscaled = cv2.resize(gray_processing, None, fx=2.0, fy=2.0, interpolation=cv2.INTER_CUBIC)
                ret, corners_up = cv2.findChessboardCorners(gray_upscaled, (nx, ny), flags)
                if ret:
                    corners = corners_up / 2.0

            if ret:
                instance_count += 1
                
                # 3. Refine Corners
                corners_refined = cv2.cornerSubPix(gray_base, corners, (11, 11), (-1, -1), subpix_criteria)
                
                # 4. Draw Corners (Small)
                cv2.drawChessboardCorners(vis_img, (nx, ny), corners_refined, ret)

                # 5. CALCULATE & DRAW BOARD CENTROID
                # Calculate mean of all corner points to find the center of the board
                cx, cy = np.mean(corners_refined, axis=0).ravel()
                center_pt = (int(cx), int(cy))

                # Draw Red Circle and Crosshair at the Centroid
                cv2.circle(vis_img, center_pt, 8, (0, 0, 255), -1) 
                cv2.drawMarker(vis_img, center_pt, (0, 255, 255), cv2.MARKER_CROSS, 15, 2)
                
                # Write Coordinates
                coord_text = f"{name}#{instance_count} ({cx:.1f}, {cy:.1f})"
                cv2.putText(vis_img, coord_text, (center_pt[0] + 15, center_pt[1]), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

                print(f"[{display_prefix}] {name} #{instance_count} Centroid: x={cx:.3f}, y={cy:.3f}")

                # 6. MASKING (Remove this board to find others)
                corners_int = corners.astype(np.int32)
                hull = cv2.convexHull(corners_int)
                mask = np.zeros_like(gray_processing)
                cv2.fillConvexPoly(mask, hull, 255)
                mask_dilated = cv2.dilate(mask, np.ones((15, 15), np.uint8))
                gray_processing[mask_dilated > 0] = 0
            else:
                break

    return vis_img

# ---------------------------------------------------------------------------
# 3. MAIN EXECUTION
# ---------------------------------------------------------------------------

if __name__ == "__main__":
    left_files = get_image_files(LEFT_IMG_PATH)
    right_files = get_image_files(RIGHT_IMG_PATH)

    if not left_files or not right_files:
        print("Error: Images not found.")
        exit()

    # Process only FIRST pair
    f_left = left_files[0]
    f_right = right_files[0]
    
    print(f"Processing: {os.path.basename(f_left)}")

    imgL = cv2.imread(f_left)
    imgR = cv2.imread(f_right)

    if imgL is not None and imgR is not None:
        visL = find_and_draw_centroids(imgL, "LEFT")
        visR = find_and_draw_centroids(imgR, "RIGHT")

        combined = np.hstack((visL, visR))
        
        if VISUALIZE:
            # Resize for screen
            scale = 0.7
            combined_resized = cv2.resize(combined, None, fx=scale, fy=scale)
            cv2.imshow('Board Centroids', combined_resized)
            
            # Wait indefinitely so you can see the result (press any key to close)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

Processing: 0000000000.png
[LEFT] boardBig #1 Centroid: x=229.884, y=231.139
[LEFT] boardBig #2 Centroid: x=423.961, y=244.631
[LEFT] boardMed #1 Centroid: x=858.465, y=177.325
[LEFT] boardMed #2 Centroid: x=559.499, y=333.820
[LEFT] boardMed #3 Centroid: x=582.619, y=435.060
[LEFT] boardMed #4 Centroid: x=1073.381, y=418.938
[LEFT] boardMed #5 Centroid: x=1116.601, y=101.568
[LEFT] boardMed #6 Centroid: x=547.198, y=105.201
[LEFT] boardMed #7 Centroid: x=849.449, y=359.824
[LEFT] boardMed #8 Centroid: x=727.906, y=356.101
[LEFT] boardMed #9 Centroid: x=1142.344, y=312.397
[LEFT] boardMed #10 Centroid: x=1355.846, y=267.604
[RIGHT] boardBig #1 Centroid: x=161.503, y=242.508
[RIGHT] boardBig #2 Centroid: x=356.066, y=252.095
[RIGHT] boardMed #1 Centroid: x=762.952, y=179.949
[RIGHT] boardMed #2 Centroid: x=487.677, y=335.474
[RIGHT] boardMed #3 Centroid: x=489.308, y=430.727
[RIGHT] boardMed #4 Centroid: x=963.158, y=406.044
[RIGHT] boardMed #5 Centroid: x=987.876, y=98.700
[RIGHT] boar